In [74]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)   #챗모델(Human, AI) 을 위한 거라면 return_messages=True 로 
memory.save_context({"input":"Hi"},{"output":"How are you ?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi'), AIMessage(content='How are you ?')]}

In [75]:

memory.save_context({"input":"Hi"},{"output":"How are you ?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi'),
  AIMessage(content='How are you ?'),
  HumanMessage(content='Hi'),
  AIMessage(content='How are you ?')]}

In [76]:
memory.save_context({"input":"Hi"},{"output":"How are you ?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi'),
  AIMessage(content='How are you ?'),
  HumanMessage(content='Hi'),
  AIMessage(content='How are you ?'),
  HumanMessage(content='Hi'),
  AIMessage(content='How are you ?')]}

In [77]:
from langchain.memory import ConversationBufferWindowMemory  #특정 메모리만 사이즈를 정해서 저장 (예:최근 5개의 대화 기록만 저장)

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4
)

def add_messages(input,output):
    memory.save_context({"input":input},{"output":output})

add_messages(1,1)


In [78]:
add_messages(2,2)
add_messages(3,3)
add_messages(4,4)

In [79]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='1'),
  AIMessage(content='1'),
  HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4')]}

In [80]:
add_messages(5,5)

In [81]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}

In [82]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)

def add_messages(input,output):
    memory.save_context({"input":input},{"output":output})

def get_history():
    return memory.load_memory_variables({})

add_messages("Hi, I'm sunny , I live in Korea","Wow, That's so cool !")

In [83]:
add_messages("Korea is very beautiful country","I wish I could go!")

In [84]:
get_history()

{'history': "The human introduces themselves as Sunny from Korea. The AI responds by expressing admiration for Sunny's location and wishes it could go visit Korea because it is a very beautiful country."}

In [85]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,
    return_messages=True, 
)

def add_messages(input,output):
    memory.save_context({"input":input},{"output":output})

def get_history():
    return memory.load_memory_variables({})

add_messages("Hi, I'm sunny , I live in Korea","Wow, That's so cool !")

In [86]:
get_history()

{'history': [HumanMessage(content="Hi, I'm sunny , I live in Korea"),
  AIMessage(content="Wow, That's so cool !")]}

In [87]:
add_messages("Korea is very beautiful country","I wish I could go!")

In [88]:
get_history()

{'history': [HumanMessage(content="Hi, I'm sunny , I live in Korea"),
  AIMessage(content="Wow, That's so cool !"),
  HumanMessage(content='Korea is very beautiful country'),
  AIMessage(content='I wish I could go!')]}

In [89]:
add_messages("How far is France from Argentina ?","I don't know. Super far!")

In [90]:
get_history()

{'history': [HumanMessage(content="Hi, I'm sunny , I live in Korea"),
  AIMessage(content="Wow, That's so cool !"),
  HumanMessage(content='Korea is very beautiful country'),
  AIMessage(content='I wish I could go!'),
  HumanMessage(content='How far is France from Argentina ?'),
  AIMessage(content="I don't know. Super far!")]}

In [91]:

from langchain.memory import ConversationKGMemory  # KG : Knowledge Graph , Entity 기준으로 정보를 만들어 감 
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)

def add_messages(input,output):
    memory.save_context({"input":input},{"output":output})

add_messages("Hi, I'm Sunny , I live in Korea.","Wow, Nice to see you!")



In [92]:
memory.load_memory_variables({"input":"who is Sunny"})

{'history': [SystemMessage(content='On Sunny: Sunny is human. Sunny lives in Korea.')]}

In [93]:
add_messages("Sunny likes Bibimbaba.","Wow, That looks Yumm!")

In [94]:
memory.load_memory_variables({"input":"what does Sunny like ?"})

{'history': [SystemMessage(content='On Sunny: Sunny is human. Sunny lives in Korea. Sunny likes Bibimbaba.')]}

In [95]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import LLMChain    #off-the-shelf LLM: 자동으로 LLM 으로  응답값을 가져오고 메모리를 업데이트 하기 위해 (한마디로 기존 템플릿을 쉽게 갖다 쓰는 것) 
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit=200,
    memory_key="chat_history",
    return_messages=True,
)

template = """
    You are a helpful AI talking to human .

    {chat_history}
    Human:{question}
    You: 

"""

prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful AI talking to human."),
    MessagesPlaceholder(variable_name="chat_history"),    
    ("human","{question}")
    ])

chain = LLMChain(
    llm=llm,
    memory=memory,
    #prompt=PromptTemplate.from_template(template),
    prompt=prompt,
    verbose=True,

)

chain.predict(question="My name is Sunny.")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to human.
Human: My name is Sunny.

> Finished chain.


'Hello Sunny! How can I assist you today?'

In [96]:
chain.predict(question="I live in Seoul.")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to human.
Human: My name is Sunny.
AI: Hello Sunny! How can I assist you today?
Human: I live in Seoul.

> Finished chain.


"That's great! Seoul is a vibrant city with a rich history and culture. Is there anything specific you'd like to know or talk about regarding Seoul?"

In [97]:
chain.predict(question="What is name ?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to human.
Human: My name is Sunny.
AI: Hello Sunny! How can I assist you today?
Human: I live in Seoul.
AI: That's great! Seoul is a vibrant city with a rich history and culture. Is there anything specific you'd like to know or talk about regarding Seoul?
Human: What is name ?

> Finished chain.


"Your name is Sunny. It's a lovely name! Is there anything else you would like to know or discuss?"

In [98]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='My name is Sunny.'),
  AIMessage(content='Hello Sunny! How can I assist you today?'),
  HumanMessage(content='I live in Seoul.'),
  AIMessage(content="That's great! Seoul is a vibrant city with a rich history and culture. Is there anything specific you'd like to know or talk about regarding Seoul?"),
  HumanMessage(content='What is name ?'),
  AIMessage(content="Your name is Sunny. It's a lovely name! Is there anything else you would like to know or discuss?")]}

In [102]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit=200,
    return_messages=True,
)


prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful AI talking to human."),
    MessagesPlaceholder(variable_name="history"),    
    ("human","{question}")
    ])

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm   # 프롬프트가 포맷되는 체인 실행전에 함수를 실행시키는 방법 

def invoke_chain(question):
    result = chain.invoke(
        {
        "question": "My name is Sunny "
        }
    )
    memory.save_context({"input":question},{"output":result.content})
    print(result)



In [103]:
invoke_chain("My name is Nico.")

content='Hello Sunny! How can I assist you today?'


In [104]:
invoke_chain("What is my name")

content='I apologize for the mistake, Sunny. How can I assist you today?'
